# Episode Event to OMOP

In [ ]:
import mysql.connector
import pandas as pd
import psycopg2
import random
import numpy as np
from datetime import datetime, timedelta

In [ ]:
df_Episode_Event_IDEA4RC = pd.read_csv("./IDEA4RC-data/episodeEventIDEA4RC.csv")
df_Episode_Event_IDEA4RC.head(5)

In [ ]:
# Conexión a la base de datos OMOP CDM
conn = psycopg2.connect(
    dbname="omopdb",
    user="postgres",
    password="mysecretpassword",
    host="localhost",
    port="5432"
)

# Comando para hacer el mapeo (insercción de datos del IDEA4RC.csv a OMOP)
cur = conn.cursor()

config = {
    'user': 'user', 
    'password': 'password',
    'host': '127.0.0.1',
    'database': 'idea4rc_dm',
    'raise_on_warnings': True
}

conn2 = mysql.connector.connect(**config)
IDEA = conn2.cursor()

### Episode Event to Episode

This Episode will be a child episode, so we can track the cancer's progression in OMOP.

episode_object_concept_id???

In [ ]:
sql = """
    INSERT INTO omopcdm.episode (person_id, episode_start_date, episode_concept_id, episode_object_concept_id, episode_type_concept_id, episode_parent_id)
    VALUES (%s, %s, %s, %s, %s, %s)
"""
query="""
    SELECT person_id
    FROM cancer_episode
    WHERE id = %s
"""
queryDate="""
    SELECT episode_start_date
    FROM cancer_episode
    WHERE id = %s
"""
sqlGetEpisode = """
    SELECT c.id 
    FROM episode c
    WHERE c.person_id = %s
    AND c.episode_start_date = %s
    LIMIT 1
"""
#Episode concept id=vocabulary from event type
episode_object_concept_id=0
event_type_concept_id=32879      #Is this ok?
df_tables=df_Episode_Event_IDEA4RC
for idx, row in df_tables.iterrows():
    curIDEA.execute(query, (row['Cancer Episode Reference']))
    res=curIDEA.fetchone()
    person_id=res
    episode_id=row['Disease status']
    episode_concept_id=row['Event type']
    episode_start_date=row['Date of episode']
    curIDEA.execute(queryDate, (row['Cancer Episode Reference']))
    res=curIDEA.fetchone()
    dateAux=res
    cur.execute(sqlGetEpisode, (person_id,dateAux))
    episode_parent_id=cur.fetchone()
    cur.execute(sql, (person_id, episode_start_date, episode_concept_id, episode_object_concept_id, event_type_concept_id, episode_parent_id))
    cur.commit()
    

### Episode Event to Procedure Ocurrance

Using 0 as the value for those columns we do not understand how to manage

In [ ]:
procedure_concept_id=0
procedure_type_concept_id=0      #This one should be the reason why the diagnosis changed or how they discovered it, I believe, yet we don't have that information in IDEA4RC
sql = """
    INSERT INTO omopcdm.procedure_occurrence (procedure_id, person_id, procedure_concept_id, procedure_date,procedure_type_concept_id,provider_id)
    VALUES (%s, %s, %s, %s, %s,%s)
"""

def episode_event_definition(idEpisode, idProcedure):
    sql = """
        INSERT INTO omopcdm.episode_event (episode_id, event_id, episode_event_field_concept_id )
        VALUES (%s, %s, %s)
    """
    cur.execute(sql, (idEpisode, idCondition,1147129))
    conn.commit()


for idx, row in df_tables.iterrows():
    curIDEA.execute(query, (row['Cancer Episode Reference']))
    res=curIDEA.fetchone()
    person_id=res
    procedure_date=row['Date of episode']
    provider_id=row['Defined At']
    curIDEA.execute(queryDate, (row['Cancer Episode Reference']))
    res=curIDEA.fetchone()
    dateAux=res
    idProcedure=datetime.now().strftime("%Y%m%d%H%M%S")
    cur.execute(sqlGetEpisode, (person_id,dateAux))
    episode_parent_id=cur.fetchone()
    episode_event_definition(episode_parent_id, idProcedure)
    cur.execute(sql, (idProcedure, person_id, procedure_concept_id, procedure_date,procedure_type_concept_id,provider_id))
    conn.commit()